In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn as nn
from torch.autograd import Variable
import pandas as pd
import os,sys
import io
from PIL import Image
import tensorflow as tf
from IPython.display import clear_output
import tensorflow_datasets as tfds
import tensorflow.io as tfio
from glob import glob
import time
import datetime
from tensorflow import keras
from tensorflow.keras import layers
#from keras.models import *
#from keras.layers import *

%load_ext autoreload
%autoreload 2

AUTOTUNE = tf.data.experimental.AUTOTUNE
print(f"Tensorflow ver. {tf.__version__}")

#https://www.tensorflow.org/tutorials/images/segmentation

Tensorflow ver. 2.4.0


In [7]:
IMG_WIDTH = 400
IMG_HEIGHT = 400
IMG_CHANNELS = 3

inputs = tf.keras.layers.Input((IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS))

#contraction
s = tf.keras.layers.Lambda(lambda x: x/255)(inputs)
c1 = tf.keras.layers.Conv2D(50,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(50,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2,2))(c1)


c2 = tf.keras.layers.Conv2D(100,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(100,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2,2))(c2)

c3 = tf.keras.layers.Conv2D(200,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.1)(c3)
c3 = tf.keras.layers.Conv2D(200,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2,2))(c3)

c4 = tf.keras.layers.Conv2D(400,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.1)(c4)
c4 = tf.keras.layers.Conv2D(400,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(c4)
p4= tf.keras.layers.MaxPooling2D((2,2))(c4)

c5 = tf.keras.layers.Conv2D(800,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.1)(c5)
c5 = tf.keras.layers.Conv2D(800,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(c5)

#decontraction

u6 = tf.keras.layers.Conv2DTranspose(400,(2,2),strides=(2,2),padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6,c4])
c6 =  tf.keras.layers.Conv2D(400,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(400,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(c6)

u7 = tf.keras.layers.Conv2DTranspose(200,(2,2),strides=(2,2),padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7,c3])
c7 =  tf.keras.layers.Conv2D(200,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(200,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(c7)

u8 = tf.keras.layers.Conv2DTranspose(100,(2,2),strides=(2,2),padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8,c2])
c8 =  tf.keras.layers.Conv2D(100,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.2)(c8)
c8 = tf.keras.layers.Conv2D(100,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(50,(2,2),strides=(2,2),padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9,c1])
c9 =  tf.keras.layers.Conv2D(50,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.2)(c9)
c9 = tf.keras.layers.Conv2D(50,(3,3),activation='relu',kernel_initializer = 'he_normal',padding='same')(c9)

outputs = tf.keras.layers.Conv2D(1,(1,1),activation='sigmoid')(c9)

model = tf.keras.Model(inputs=[inputs],outputs=[outputs])
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt,loss = 'binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 400, 400, 3) 0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 400, 400, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 400, 400, 50) 1400        lambda_3[0][0]                   
__________________________________________________________________________________________________
dropout_27 (Dropout)            (None, 400, 400, 50) 0           conv2d_57[0][0]                  
____________________________________________________________________________________________

In [15]:
SEED = 42

dataset_path = "training/" 
training_data = "images/"
val_data = "test_set/"

# Image size that we are going to use
IMG_SIZE = 400
# Our images are RGB (3 channels)
N_CHANNELS = 3
# Scene Parsing has 150 classes + `not labeled`
N_CLASSES = 2

TRAINSET_SIZE = len(glob(dataset_path + training_data + "*.png"))
print(f"The Training Dataset contains {TRAINSET_SIZE} images.")
print(dataset_path + training_data + "*.png")


Groundtruth_SIZE = len(glob(dataset_path + "groundtruth/" + "*.png"))
print(f"The Training Dataset contains {TRAINSET_SIZE} images.")
print(dataset_path + training_data + "*.png")



VALSET_SIZE = len(glob(val_data +training_data+ "*.png"))
print(f"The Validation Dataset contains {VALSET_SIZE} images.")

from helper_unet import parse_image

train_dataset = tf.data.Dataset.list_files(dataset_path + training_data + "*.png", seed=SEED)
train_dataset = train_dataset.map(parse_image)
print(len(train_dataset))

val_dataset = tf.data.Dataset.list_files(val_data + training_data+"*.png", seed=SEED)
val_dataset =val_dataset.map(parse_image)
print(len(val_dataset))

from helper_unet import load_image_test, load_image_train

BATCH_SIZE = 32

BUFFER_SIZE = 1000

dataset = {"train": train_dataset, "val": val_dataset}

# -- Train Dataset --#
dataset['train'] = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset['train'] = dataset['train'].shuffle(buffer_size=BUFFER_SIZE, seed=SEED)
dataset['train'] = dataset['train'].repeat()
dataset['train'] = dataset['train'].batch(BATCH_SIZE)
dataset['train'] = dataset['train'].prefetch(buffer_size=AUTOTUNE)

#-- Validation Dataset --#
dataset['val'] = dataset['val'].map(load_image_test)
dataset['val'] = dataset['val'].repeat()
dataset['val'] = dataset['val'].batch(BATCH_SIZE)
dataset['val'] = dataset['val'].prefetch(buffer_size=AUTOTUNE)

print(dataset['train'])
print(dataset['val'])



The Training Dataset contains 100 images.
training/images/*.png
The Training Dataset contains 100 images.
training/images/*.png
The Validation Dataset contains 50 images.
100
50
<PrefetchDataset shapes: ((None, 400, 400, 3), (None, 400, 400, 1)), types: (tf.float32, tf.float32)>
<PrefetchDataset shapes: ((None, 400, 400, 3), (None, 400, 400, 1)), types: (tf.float32, tf.float32)>


In [16]:
from helper_unet import create_mask, show_predictions


for image, mask in dataset['train'].take(2):
    sample_image, sample_mask = image, mask
    one_img_batch = sample_image[0][tf.newaxis, ...]
    inference = model.predict(one_img_batch)
    pred_mask = create_mask(inference)
    #display_sample([sample_image[0], sample_mask[0],
                        #pred_mask[0]])



In [17]:
EPOCHS = 1

STEPS_PER_EPOCH = TRAINSET_SIZE // BATCH_SIZE
VALIDATION_STEPS = VALSET_SIZE // BATCH_SIZE

class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

model_history = model.fit(dataset['train'], epochs=EPOCHS,
steps_per_epoch=STEPS_PER_EPOCH,
validation_steps=VALIDATION_STEPS,
validation_data=dataset['val'], callbacks=[DisplayCallback()])

3/3 [==============================] - ETA: 0s - loss: 0.5689 - accuracy: 0.0795  

NotFoundError:  test_set/groundtruth/test_50.png; No such file or directory
	 [[{{node ReadFile_1}}]]
	 [[IteratorGetNext]] [Op:__inference_test_function_6223]

Function call stack:
test_function
